# Import + CSV

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import prince

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

train = pd.read_csv('./train.csv').drop(columns=['ID'])
test = pd.read_csv('./test.csv').drop(columns=['ID'])

# EDA

### 단순 drop (수정: 그냥 결측치 많은 거 전부 제거 + 난자 채취 경과일 살리기)

In [5]:
train.drop(columns=['임신 시도 또는 마지막 임신 경과 연수'], inplace=True)
test.drop(columns=['임신 시도 또는 마지막 임신 경과 연수'], inplace=True)

train.drop(columns=['착상 전 유전 검사 사용 여부'], inplace=True)
test.drop(columns=['착상 전 유전 검사 사용 여부'], inplace=True)

train.drop(columns=['난자 해동 경과일'], inplace=True)
test.drop(columns=['난자 해동 경과일'], inplace=True)

train.drop(columns=['배아 해동 경과일'], inplace=True)
test.drop(columns=['배아 해동 경과일'], inplace=True)

train.drop(columns=['PGS 시술 여부'], inplace=True)
test.drop(columns=['PGS 시술 여부'], inplace=True)

train.drop(columns=['PGD 시술 여부'], inplace=True)
test.drop(columns=['PGD 시술 여부'], inplace=True)

train.drop(columns=['동결 배아 사용 여부'], inplace=True)
test.drop(columns=['동결 배아 사용 여부'], inplace=True)

train.drop(columns=['난자 채취 경과일'], inplace=True)
test.drop(columns=['난자 채취 경과일'], inplace=True)

train.drop(columns=['난자 혼합 경과일'], inplace=True)
test.drop(columns=['난자 혼합 경과일'], inplace=True)

train.drop(columns=['배아 이식 경과일'], inplace=True)
test.drop(columns=['배아 이식 경과일'], inplace=True)

### DI 시술 여부일 때 결측치 -1 처리

In [7]:
train.drop(columns=['미세주입된 난자 수'], inplace=True)
test.drop(columns=['미세주입된 난자 수'], inplace=True)

train.drop(columns=['미세주입 배아 이식 수'], inplace=True)
test.drop(columns=['미세주입 배아 이식 수'], inplace=True)

train.drop(columns=['미세주입 후 저장된 배아 수'], inplace=True)
test.drop(columns=['미세주입 후 저장된 배아 수'], inplace=True)

train.drop(columns=['미세주입에서 생성된 배아 수'], inplace=True)
test.drop(columns=['미세주입에서 생성된 배아 수'], inplace=True)

train.drop(columns=['혼합된 난자 수'], inplace=True)
test.drop(columns=['혼합된 난자 수'], inplace=True)

train.drop(columns=['파트너 정자와 혼합된 난자 수'], inplace=True)
test.drop(columns=['파트너 정자와 혼합된 난자 수'], inplace=True)

train.drop(columns=['기증자 정자와 혼합된 난자 수'], inplace=True)
test.drop(columns=['기증자 정자와 혼합된 난자 수'], inplace=True)

features = ['총 생성 배아 수', '이식된 배아 수', '저장된 배아 수',
            '해동된 배아 수', '해동 난자 수', '수집된 신선 난자 수', '저장된 신선 난자 수']

train[features] = train[features].fillna(-1)
test[features] = test[features].fillna(-1)

### DI 시술 여부일 때 결측치 -1 처리 후 원 핫 인코딩 (추가됨)

In [9]:
# 결측치를 -1로 채우기
cols_to_fill = ['신선 배아 사용 여부', '기증 배아 사용 여부', '단일 배아 이식 여부']

train[cols_to_fill] = train[cols_to_fill].fillna(-1)

train.drop(columns=['대리모 여부'], inplace=True)
test.drop(columns=['대리모 여부'], inplace=True)

train.drop(columns=['착상 전 유전 진단 사용 여부'], inplace=True)
test.drop(columns=['착상 전 유전 진단 사용 여부'], inplace=True)

# 원핫인코딩 적용
train_encoded = pd.get_dummies(train, columns=cols_to_fill)

# 기존 열을 삭제한 train 데이터프레임
train = train_encoded

# 결측치를 -1로 채우기
cols_to_fill = ['신선 배아 사용 여부', '기증 배아 사용 여부', '단일 배아 이식 여부']

test[cols_to_fill] = test[cols_to_fill].fillna(-1)

# 원핫인코딩 적용
test_encoded = pd.get_dummies(test, columns=cols_to_fill)

# 기존 열을 삭제한 test 데이터프레임
test = test_encoded


### 주/부 count feature 추가

In [11]:
train["주 불임 원인 개수"] = (
    train["남성 주 불임 원인"] + train["여성 주 불임 원인"] + train["부부 주 불임 원인"]
)
test["주 불임 원인 개수"] = (
    test["남성 주 불임 원인"] + test["여성 주 불임 원인"] + test["부부 주 불임 원인"]
)

train["부 불임 원인 개수"] = (
    train["남성 부 불임 원인"] + train["여성 부 불임 원인"] + train["부부 부 불임 원인"]
)
test["부 불임 원인 개수"] = (
    test["남성 부 불임 원인"] + test["여성 부 불임 원인"] + test["부부 부 불임 원인"]
)

train.drop(columns=['남성 주 불임 원인'], inplace=True)
test.drop(columns=['남성 주 불임 원인'], inplace=True)

train.drop(columns=['남성 부 불임 원인'], inplace=True)
test.drop(columns=['남성 부 불임 원인'], inplace=True)

train.drop(columns=['여성 주 불임 원인'], inplace=True)
test.drop(columns=['여성 주 불임 원인'], inplace=True)

train.drop(columns=['여성 부 불임 원인'], inplace=True)
test.drop(columns=['여성 부 불임 원인'], inplace=True)

train.drop(columns=['부부 주 불임 원인'], inplace=True)
test.drop(columns=['부부 주 불임 원인'], inplace=True)

train.drop(columns=['부부 부 불임 원인'], inplace=True)
test.drop(columns=['부부 부 불임 원인'], inplace=True)

### 여성 요인 합체 (이건 원본에도 반영해주자)

In [13]:
train["불임 원인 - 여성 요인"] = (
    train["불임 원인 - 자궁내막증"] + train["불임 원인 - 자궁경부 문제"] + train["불임 원인 - 난관 질환"] + train["불임 원인 - 배란 장애"]
)
test["불임 원인 - 여성 요인"] = (
    test["불임 원인 - 자궁내막증"] + test["불임 원인 - 자궁경부 문제"] + test["불임 원인 - 난관 질환"] + test["불임 원인 - 배란 장애"]
)

train.drop(columns=['불임 원인 - 자궁내막증'], inplace=True)
test.drop(columns=['불임 원인 - 자궁내막증'], inplace=True)

train.drop(columns=['불임 원인 - 자궁경부 문제'], inplace=True)
test.drop(columns=['불임 원인 - 자궁경부 문제'], inplace=True)

train.drop(columns=['불임 원인 - 난관 질환'], inplace=True)
test.drop(columns=['불임 원인 - 난관 질환'], inplace=True)

train.drop(columns=['불임 원인 - 배란 장애'], inplace=True)
test.drop(columns=['불임 원인 - 배란 장애'], inplace=True)

### 남성 요인 개수 feature 추가

In [15]:
train["불임 원인 - 남성 요인 개수"] = (
    train["불임 원인 - 정자 형태"] +
    train["불임 원인 - 정자 운동성"] +
    train["불임 원인 - 정자 면역학적 요인"] +
    train["불임 원인 - 정자 농도"]
)

test["불임 원인 - 남성 요인 개수"] = (
    test["불임 원인 - 정자 형태"] +
    test["불임 원인 - 정자 운동성"] +
    test["불임 원인 - 정자 면역학적 요인"] +
    test["불임 원인 - 정자 농도"]
)

train.drop(columns=["불임 원인 - 정자 형태"], inplace=True)
test.drop(columns=["불임 원인 - 정자 형태"], inplace=True)

train.drop(columns=['불임 원인 - 정자 운동성'], inplace=True)
test.drop(columns=['불임 원인 - 정자 운동성'], inplace=True)

train.drop(columns=['불임 원인 - 정자 면역학적 요인'], inplace=True)
test.drop(columns=['불임 원인 - 정자 면역학적 요인'], inplace=True)

train.drop(columns=['불임 원인 - 정자 농도'], inplace=True)
test.drop(columns=['불임 원인 - 정자 농도'], inplace=True)

### 배아 생성 주요 이유 (원핫)

In [17]:
train.drop(columns=['배아 생성 주요 이유'], inplace=True)
test.drop(columns=['배아 생성 주요 이유'], inplace=True)

### 시술 횟수 관련

In [19]:
valid_values = ['0회', '1회', '2회', '3회', '4회', '5회', '6회 이상']

## train ##
train = train[
    (train['총 시술 횟수'].isin(valid_values)) &
    (train['클리닉 내 총 시술 횟수'].isin(valid_values)) &
    (train['IVF 시술 횟수'].isin(valid_values)) &
    (train['DI 시술 횟수'].isin(valid_values)) &
    (train['총 출산 횟수'].isin(valid_values)) &
    (train['IVF 출산 횟수'].isin(valid_values)) &
    (train['DI 출산 횟수'].isin(valid_values)) &
    (train['총 임신 횟수'].isin(valid_values)) &
    (train['IVF 임신 횟수'].isin(valid_values)) &
    (train['DI 임신 횟수'].isin(valid_values))
]

for col in ['총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수']:
    train[col] = train[col].replace('6회 이상', '6')
    train[col] = train[col].str.replace('회', '', regex=True).astype(int)

train['총 시술 횟수'] = train['IVF 시술 횟수'] + train['DI 시술 횟수']

## test ##
test = test[
    (test['총 시술 횟수'].isin(valid_values)) &
    (test['클리닉 내 총 시술 횟수'].isin(valid_values)) &
    (test['IVF 시술 횟수'].isin(valid_values)) &
    (test['DI 시술 횟수'].isin(valid_values)) &
    (test['총 출산 횟수'].isin(valid_values)) &
    (test['IVF 출산 횟수'].isin(valid_values)) &
    (test['DI 출산 횟수'].isin(valid_values)) &
    (test['총 임신 횟수'].isin(valid_values)) &
    (test['IVF 임신 횟수'].isin(valid_values)) &
    (test['DI 임신 횟수'].isin(valid_values))
]

for col in ['총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수']:
    test[col] = test[col].replace('6회 이상', '6')
    test[col] = test[col].str.replace('회', '', regex=True).astype(int)

test['총 시술 횟수'] = test['IVF 시술 횟수'] + test['DI 시술 횟수']

train.drop(columns=['클리닉 내 총 시술 횟수'], inplace=True)
test.drop(columns=['클리닉 내 총 시술 횟수'], inplace=True)

train.drop(columns=['IVF 시술 횟수'], inplace=True)
test.drop(columns=['IVF 시술 횟수'], inplace=True)

train.drop(columns=['DI 시술 횟수'], inplace=True)
test.drop(columns=['DI 시술 횟수'], inplace=True)

train.drop(columns=['DI 임신 횟수'], inplace=True)
test.drop(columns=['DI 임신 횟수'], inplace=True)

train.drop(columns=['IVF 임신 횟수'], inplace=True)
test.drop(columns=['IVF 임신 횟수'], inplace=True)

train.drop(columns=['DI 출산 횟수'], inplace=True)
test.drop(columns=['DI 출산 횟수'], inplace=True)

train.drop(columns=['IVF 출산 횟수'], inplace=True)
test.drop(columns=['IVF 출산 횟수'], inplace=True)

### 특정 시술 유형 그룹화 + 원핫

In [21]:
train.drop(columns=['특정 시술 유형'], inplace=True)
test.drop(columns=['특정 시술 유형'], inplace=True)

### 배란 유도 유형 정상화

In [23]:
train.drop(columns=['배란 유도 유형'], inplace=True)
test.drop(columns=['배란 유도 유형'], inplace=True)

### 난자출처/정자출처 원핫인코딩

In [25]:
train.drop(columns=['난자 출처'], inplace=True)
test.drop(columns=['난자 출처'], inplace=True)

train.drop(columns=['정자 출처'], inplace=True)
test.drop(columns=['정자 출처'], inplace=True)

### 난자/정자 기증자 나이 라벨 인코딩 (수정: 오타 해결 + 결측치 -1 처리)

In [27]:
from sklearn.preprocessing import LabelEncoder

# 결측치를 -1로 처리 (난자 기증자 나이, 정자 기증자 나이)
train["난자 기증자 나이"].fillna("-1", inplace=True)
test["난자 기증자 나이"].fillna("-1", inplace=True)

# 라벨 인코딩 매핑 딕셔너리
age_mapping_nanja = {
    "만20세 이하": 0,
    "만21-25세": 1,
    "만26-30세": 2,
    "만31-35세": 3,
    "알 수 없음": -1
}

# 라벨 인코딩 적용(난자)
train["난자 기증자 나이 (라벨 인코딩)"] = train["난자 기증자 나이"].map(age_mapping_nanja)
test["난자 기증자 나이 (라벨 인코딩)"] = test["난자 기증자 나이"].map(age_mapping_nanja)

# 원본 "난자 기증자 나이" 컬럼 제거(난자)
train.drop(columns=["난자 기증자 나이"], inplace=True)
test.drop(columns=["난자 기증자 나이"], inplace=True)

# 원본 "정자 기증자 나이" 컬럼 제거(정자)
train.drop(columns=["정자 기증자 나이"], inplace=True)
test.drop(columns=["정자 기증자 나이"], inplace=True)

C:\Users\USER\AppData\Local\Temp\ipykernel_8636\293370775.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train["난자 기증자 나이"].fillna("-1", inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_8636\293370775.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, whe

### 시술 당시 나이 라벨 인코딩

In [29]:
# 라벨 인코딩 매핑 딕셔너리 (나이가 작은 순으로 매핑)
age_mapping_treatment = {
    "만18-34세": 0,
    "만35-37세": 1,
    "만38-39세": 2,
    "만40-42세": 3,
    "만43-44세": 4,
    "만45-50세": 5,
    "알 수 없음": -1
}

# 라벨 인코딩 적용
train["시술 당시 나이 (라벨 인코딩)"] = train["시술 당시 나이"].map(age_mapping_treatment)
test["시술 당시 나이 (라벨 인코딩)"] = test["시술 당시 나이"].map(age_mapping_treatment)

# 원본 "시술 당시 나이" 컬럼 제거
train.drop(columns=["시술 당시 나이"], inplace=True)
test.drop(columns=["시술 당시 나이"], inplace=True)

### 시술 시기 코드 원 핫 인코딩

In [31]:
train.drop(columns=['시술 시기 코드'], inplace=True)
test.drop(columns=['시술 시기 코드'], inplace=True)

In [32]:
train.drop(columns=['배란 자극 여부'], inplace=True)
test.drop(columns=['배란 자극 여부'], inplace=True)

***

# Label/One-Hot/Numeric Column List

In [35]:
label_columns = ['불명확 불임 원인', '시술 유형']

numeric_columns = ['임신 시도 또는 마지막 임신 경과 연수', '난자 혼합 경과일', '배아 이식 경과일'
                  , '총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수'
                   , 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수' ]

# 결측치 처리 (추가됨: 결측치가 있는 남은 세 개의 정수형 feature를 중앙값으로 채우기)

# 인코딩

In [38]:
# 카테고리형 컬럼들을 문자열로 변환
for col in label_columns:
    train[col] = train[col].astype(str)
    test[col] = test[col].astype(str)

ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
train[label_columns] = ordinal_encoder.fit_transform(train[label_columns])
test[label_columns] = ordinal_encoder.transform(test[label_columns])

# X와 y로 분리

In [40]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

***

# 하이퍼파라미터 (stacking으로 바꿈, 학습모델과 파라미터는 계속 찾아야 할 듯)

In [66]:
import optuna
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import pandas as pd
from imblearn.over_sampling import SMOTE  # SMOTE 임포트

# 데이터 분할
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# SMOTE 적용
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Optuna 하이퍼파라미터 튜닝을 위한 목적 함수
def objective(trial):
    # LightGBM 하이퍼파라미터 설정
    lgb_param = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),  # 더 작은 학습률
        'num_leaves': trial.suggest_int('num_leaves', 31, 127),  # 더 넓은 범위
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),  # 더 넓은 범위
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),  # 데이터 샘플링 비율
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),  # 특성 샘플링 비율
        'max_bin': trial.suggest_int('max_bin', 100, 300),  # 최대 bin 개수 증가
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),  # 과적합 방지를 위한 최소 샘플 수
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-5, 10),  # L1 정규화
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-5, 10),  # L2 정규화
        'random_state': 42
    }

    # LightGBM 모델 학습
    lgb_clf = lgb.LGBMClassifier(**lgb_param)
    lgb_clf.fit(X_train_resampled, y_train_resampled)

    # 예측 (ROC-AUC 계산)
    y_pred = lgb_clf.predict_proba(X_valid)[:, 1]
    score = roc_auc_score(y_valid, y_pred)

    return score

# Optuna 최적화
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)  # n_trials를 100으로 늘려서 더 많은 실험을 시도

# 최적 하이퍼파라미터 출력
print(f"Best Trial: {study.best_trial.params}")
print(f"Best ROC-AUC Score: {study.best_value}")

# 최적 하이퍼파라미터로 LightGBM 모델 학습
best_lgb_params = study.best_trial.params
lgb_clf_final = lgb.LGBMClassifier(**best_lgb_params)

# 전체 데이터로 학습
lgb_clf_final.fit(X, y)

# test 데이터 예측
pred_proba = lgb_clf_final.predict_proba(test)[:, 1]

# 결과 저장
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission['probability'] = pred_proba
sample_submission.to_csv('./final_submission.csv', index=False)

print("✅ 최종 예측 완료! 결과 저장됨: final_submission.csv")

[I 2025-02-25 23:15:37,549] A new study created in memory with name: no-name-e247d0ae-6ef8-4eb2-968d-cf05abc99f2a
C:\Users\USER\AppData\Local\Temp\ipykernel_8636\4147431200.py:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),  # 더 작은 학습률
C:\Users\USER\AppData\Local\Temp\ipykernel_8636\4147431200.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-5, 10),  # L1 정규화
C:\Users\USER\AppData\Local\Temp\ipykernel_8636\4147431200.py:27: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://gi

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 151978, number of negative: 151978
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.023921 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1060
[LightGBM] [Info] Number of data points in the train set: 303956, number of used features: 28
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


[I 2025-02-25 23:15:58,182] Trial 0 finished with value: 0.6995532676253772 and parameters: {'learning_rate': 0.07346969833390633, 'num_leaves': 97, 'n_estimators': 851, 'subsample': 0.8486247279192718, 'colsample_bytree': 0.971909987838852, 'max_bin': 137, 'min_child_samples': 89, 'reg_alpha': 1.978060688563251, 'reg_lambda': 0.0006667494130268602}. Best is trial 0 with value: 0.6995532676253772.
C:\Users\USER\AppData\Local\Temp\ipykernel_8636\4147431200.py:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),  # 더 작은 학습률
C:\Users\USER\AppData\Local\Temp\ipykernel_8636\4147431200.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_fl

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 151978, number of negative: 151978
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.022429 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1303
[LightGBM] [Info] Number of data points in the train set: 303956, number of used features: 28
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


[I 2025-02-25 23:16:12,682] Trial 1 finished with value: 0.7077144864139533 and parameters: {'learning_rate': 0.037454511414079995, 'num_leaves': 54, 'n_estimators': 887, 'subsample': 0.7669405972981821, 'colsample_bytree': 0.9765130832220639, 'max_bin': 173, 'min_child_samples': 17, 'reg_alpha': 0.5413377779661189, 'reg_lambda': 0.0005718365997809682}. Best is trial 1 with value: 0.7077144864139533.
C:\Users\USER\AppData\Local\Temp\ipykernel_8636\4147431200.py:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),  # 더 작은 학습률
C:\Users\USER\AppData\Local\Temp\ipykernel_8636\4147431200.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest

...

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 151978, number of negative: 151978
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.024562 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1347
[LightGBM] [Info] Number of data points in the train set: 303956, number of used features: 28
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


[I 2025-02-25 23:26:07,238] Trial 98 finished with value: 0.7127385374624167 and parameters: {'learning_rate': 0.010410488800419714, 'num_leaves': 82, 'n_estimators': 800, 'subsample': 0.862709813146058, 'colsample_bytree': 0.8882301413253253, 'max_bin': 180, 'min_child_samples': 74, 'reg_alpha': 3.4337558414757265e-05, 'reg_lambda': 1.4347921963884998}. Best is trial 82 with value: 0.7219428381485462.
C:\Users\USER\AppData\Local\Temp\ipykernel_8636\4147431200.py:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),  # 더 작은 학습률
C:\Users\USER\AppData\Local\Temp\ipykernel_8636\4147431200.py:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use sugge

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 151978, number of negative: 151978
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.029141 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1101
[LightGBM] [Info] Number of data points in the train set: 303956, number of used features: 28
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


[I 2025-02-25 23:26:22,162] Trial 99 finished with value: 0.7131687976105515 and parameters: {'learning_rate': 0.013822961192437155, 'num_leaves': 38, 'n_estimators': 995, 'subsample': 0.828278238745106, 'colsample_bytree': 0.8571843978006639, 'max_bin': 143, 'min_child_samples': 88, 'reg_alpha': 1.301704072417137e-05, 'reg_lambda': 0.16931809334689527}. Best is trial 82 with value: 0.7219428381485462.


Best Trial: {'learning_rate': 0.010540712994132227, 'num_leaves': 34, 'n_estimators': 171, 'subsample': 0.8021482764635024, 'colsample_bytree': 0.8941596971661392, 'max_bin': 167, 'min_child_samples': 82, 'reg_alpha': 2.9977582889799054e-05, 'reg_lambda': 0.20341964486616235}
Best ROC-AUC Score: 0.7219428381485462
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 66228, number of negative: 190123
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.018550 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 303
[LightGBM] [Info] Number of data points in the train set: 256351, number of used features: 28
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258349 -> initscore=-1.054568
[LightGBM] [Info] Start training from score -1.054568
✅ 최종 예측 완료! 결과 저장됨: final_submission.csv


# 학습